In [1]:
import findspark

In [2]:
# COMMAND ----------
# Databricks notebook source
from multiprocessing import current_process
import glob
import pyspark.pandas as pf
import pandas as pd
from pathlib import Path
import warnings
import logging
import sys
import os
import time
import pyspark
from delta import *
from pyspark.conf import SparkConf

conf = SparkConf()
conf.set("spark.driver.maxResultSize", "8g")
conf.set("spark.driver.memory", "8g") 
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")


builder = pyspark.sql.SparkSession.builder.appName("dev_test_notebook") \
    .enableHiveSupport()\
    .config(conf=conf) 

spark = configure_spark_with_delta_pip(builder).getOrCreate()

/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7edfe812-a49d-4c2b-ad30-088f4d6eed8b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 141ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

23/07/12 11:57:34 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar added multiple times to distributed cache.
23/07/12 11:57:34 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/io.delta_delta-storage-2.3.0.jar added multiple times to distributed cache.
23/07/12 11:57:34 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar added multiple times to distributed cache.


In [3]:
import pandas as pd

file_path = 'file:/landing/Raw/Merlin/ecb2ef6fdf3042339961920fee90b950/ecb2ef6fdf3042339961920fee90b950/DIS_NEST_NEST_00000300_0.parquet'
read_local_file_pd = pd.read_parquet(file_path, engine='fastparquet')
read_local_file_pd.head(10)

,NstRef,ShtRef,RIndex,Quantity,Var1,Var2,Var3,Var4,Var5,RecState,CrtDate,LastDate,CrtUser,LastUser,Owner,RecEnt,RecOU,RecSec,CntID,RecID
0,Nestings\2\2038,C15N612-B3P,0,1,,,,,,0,2011-09-05 10:40:39,2011-09-05 10:40:39,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,967
1,Nestings\2\2111,chapa montajes cantabrico-B4L,0,1,,,,,,0,2011-09-09 15:26:06,2011-09-09 15:26:06,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,996
2,Nestings\2\2111,4A20N709-1,0,1,,,,,,0,2011-09-09 15:26:06,2011-09-09 15:26:06,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,997
3,Nestings\4\4538,3B20D229-242--1,0,1,,,,,,0,2011-12-07 10:11:31,2011-12-07 10:11:31,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,2429
4,Nestings\4\4554,3B8N1334-2,0,1,,,,,,0,2011-12-12 15:38:57,2011-12-12 15:38:57,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,2549
5,Nestings\5\5314,Presupuesto Inox 6-DF4,0,1,,,,,,0,2012-01-05 13:46:13,2012-01-05 13:46:13,Saul,Saul,Saul,ment,ment,0,-1,3030
6,Nestings\5\5328,presuspuesto-DFF,0,1,,,,,,0,2012-01-09 12:37:40,2012-01-09 12:37:40,Orlando,Orlando,Orlando,ment,ment,0,-1,3044
7,Nestings\5\5329,Presupuesto esp 10-DFG,0,1,,,,,,0,2012-01-09 12:40:50,2012-01-09 12:40:50,Orlando,Orlando,Orlando,ment,ment,0,-1,3045
8,Nestings\5\5330,Presupuestacion esp 6-DFH,0,1,,,,,,0,2012-01-09 12:41:47,2012-01-09 12:41:47,Orlando,Orlando,Orlando,ment,ment,0,-1,3046
9,Nestings\5\5331,presupuesto 20mm-DFJ,0,1,,,,,,0,2012-01-09 12:57:22,2012-01-09 12:57:22,Orlando,Orlando,Orlando,ment,ment,0,-1,3047


In [4]:
read_local_file_pf = pd.read_parquet(file_path, engine='fastparquet')
pf_df = pf.DataFrame(read_local_file_pf)
pf_df.head(10)


/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


23/07/12 11:58:09 WARN TaskSetManager: Stage 0 contains a task of very large size (5980 KiB). The maximum recommended task size is 1000 KiB.


/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,NstRef,ShtRef,RIndex,Quantity,Var1,Var2,Var3,Var4,Var5,RecState,CrtDate,LastDate,CrtUser,LastUser,Owner,RecEnt,RecOU,RecSec,CntID,RecID
0,Nestings\2\2038,C15N612-B3P,0,1,,,,,,0,2011-09-05 10:40:39,2011-09-05 10:40:39,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,967
1,Nestings\2\2111,chapa montajes cantabrico-B4L,0,1,,,,,,0,2011-09-09 15:26:06,2011-09-09 15:26:06,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,996
2,Nestings\2\2111,4A20N709-1,0,1,,,,,,0,2011-09-09 15:26:06,2011-09-09 15:26:06,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,997
3,Nestings\4\4538,3B20D229-242--1,0,1,,,,,,0,2011-12-07 10:11:31,2011-12-07 10:11:31,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,2429
4,Nestings\4\4554,3B8N1334-2,0,1,,,,,,0,2011-12-12 15:38:57,2011-12-12 15:38:57,SYSADM,SYSADM,SYSADM,ment,ment,0,-1,2549
5,Nestings\5\5314,Presupuesto Inox 6-DF4,0,1,,,,,,0,2012-01-05 13:46:13,2012-01-05 13:46:13,Saul,Saul,Saul,ment,ment,0,-1,3030
6,Nestings\5\5328,presuspuesto-DFF,0,1,,,,,,0,2012-01-09 12:37:40,2012-01-09 12:37:40,Orlando,Orlando,Orlando,ment,ment,0,-1,3044
7,Nestings\5\5329,Presupuesto esp 10-DFG,0,1,,,,,,0,2012-01-09 12:40:50,2012-01-09 12:40:50,Orlando,Orlando,Orlando,ment,ment,0,-1,3045
8,Nestings\5\5330,Presupuestacion esp 6-DFH,0,1,,,,,,0,2012-01-09 12:41:47,2012-01-09 12:41:47,Orlando,Orlando,Orlando,ment,ment,0,-1,3046
9,Nestings\5\5331,presupuesto 20mm-DFJ,0,1,,,,,,0,2012-01-09 12:57:22,2012-01-09 12:57:22,Orlando,Orlando,Orlando,ment,ment,0,-1,3047


In [5]:
# Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false)) error en los parquet
read_local_file_pf = pf.read_parquet(file_path,engine='fastparquet')
read_local_file_pf.head(10)

/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_parquet`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


23/07/12 11:58:28 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 1) (worker1 executor 2): org.apache.spark.sql.AnalysisException: Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false))
	at org.apache.spark.sql.errors.QueryCompilationErrors$.illegalParquetTypeError(QueryCompilationErrors.scala:1317)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.illegalType$1(ParquetSchemaConverter.scala:191)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.$anonfun$convertPrimitiveField$2(ParquetSchemaConverter.scala:269)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.convertPrimitiveField(ParquetSchemaConverter.scala:209)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.convertField(ParquetSchemaConverter.scala:173)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.$anonfun

Py4JJavaError: An error occurred while calling o232.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 4) (worker2 executor 1): org.apache.spark.sql.AnalysisException: Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false))
	at org.apache.spark.sql.errors.QueryCompilationErrors$.illegalParquetTypeError(QueryCompilationErrors.scala:1317)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.illegalType$1(ParquetSchemaConverter.scala:191)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.$anonfun$convertPrimitiveField$2(ParquetSchemaConverter.scala:269)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.convertPrimitiveField(ParquetSchemaConverter.scala:209)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConve

In [6]:
# read file in the hdbs if is local file:/
file_path = 'file:/landing/Staging/Merlin/Smartquoting/3cb9a027-3b3e-4f3d-b9df-d8478cf812c8/8abe3f95-37f1-4cbc-a152-52e79ed26a65/staging_db_data_production.csv'
read_local_file_pf = pf.read_csv(file_path,sep=";",
                         decimal=',',
                         encoding='ISO-8859-1',
                         quotechar='"',
                         engine='python',
                         dtype=str,
                         na_values='nan',
                         inferSchema=True)


/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [7]:
read_local_file_pf = read_local_file_pf[["Part_IsAssembly", "Part_Weight", "Part_RealArea", "Part_ExternalArea", "Part_Thickness", "Part_Length", "Part_RectangularArea",
            "Part_IsFillerPart", "Part_MaterialDensity", "Part_RectangularWeight", "Part_MarkingPerimeter","Part_ExternalWeight", "Part_CutPerimeter",
            "Part_Width", "ManufacturingOrder_Quantity", "Part_WorkCenter", "Part_Material", "Part_Rotations", "Part_CreationMethod", "Tenant_ID",
            "Plant_ID"]]
read_local_file_spf = read_local_file_pf.to_spark()
display(read_local_file_spf.head(10))

/home/jupyter/.conda/envs/pyspark_code_env/lib/python3.8/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


[Row(Part_IsAssembly='0', Part_Weight='5.38824', Part_RealArea='0.0572', Part_ExternalArea='0.0586', Part_Thickness='12.0', Part_Length='254.5', Part_RectangularArea='0.0597', Part_IsFillerPart='0', Part_MaterialDensity='7.85', Part_RectangularWeight='5.6219', Part_MarkingPerimeter='0.0', Part_ExternalWeight='5.52012', Part_CutPerimeter='1.2998', Part_Width='234.5', ManufacturingOrder_Quantity='25.0', Part_WorkCenter='Prima CH-9000L', Part_Material='MS', Part_Rotations='1.111111111E49', Part_CreationMethod='DXF', Tenant_ID='3cb9a027-3b3e-4f3d-b9df-d8478cf812c8', Plant_ID='8abe3f95-37f1-4cbc-a152-52e79ed26a65'),
 Row(Part_IsAssembly='0', Part_Weight='4.6817400000000005', Part_RealArea='0.0497', Part_ExternalArea='0.052', Part_Thickness='12.0', Part_Length='224.5', Part_RectangularArea='0.0531', Part_IsFillerPart='0', Part_MaterialDensity='7.85', Part_RectangularWeight='5.0015', Part_MarkingPerimeter='0.0', Part_ExternalWeight='4.8984', Part_CutPerimeter='1.3305', Part_Width='236.5', Man

In [8]:
# Write the DataFrame to a Delta table in Hive
hive_table_name = 'default.cr_table_demo_delta'
hive_table_path= f'/user/jupyter/result/{hive_table_name}'

# write table 
read_local_file_spf.write.format("delta").mode('overwrite').save(hive_table_path)



23/07/12 12:07:54 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
# Get the schema of the DataFrame
schema = read_local_file_spf.schema
# Create the table in Hive
column_definitions = ", ".join([f"{field.name} {field.dataType.simpleString()}" for field in schema])
print(column_definitions)
# Create an external Hive table pointing to the Delta table location
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {hive_table_name} ({column_definitions}) USING delta LOCATION '{hive_table_path}'")

Part_IsAssembly string, Part_Weight string, Part_RealArea string, Part_ExternalArea string, Part_Thickness string, Part_Length string, Part_RectangularArea string, Part_IsFillerPart string, Part_MaterialDensity string, Part_RectangularWeight string, Part_MarkingPerimeter string, Part_ExternalWeight string, Part_CutPerimeter string, Part_Width string, ManufacturingOrder_Quantity string, Part_WorkCenter string, Part_Material string, Part_Rotations string, Part_CreationMethod string, Tenant_ID string, Plant_ID string
23/07/12 12:09:12 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `default`.`cr_table_demo_delta` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [10]:
# Create an external Hive table pointing to the Delta table location
hive_table_name = 'default.cr_table_demo_hive'
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {hive_table_name} ({column_definitions}) USING hive LOCATION '{hive_table_path}'")

DataFrame[]

In [11]:
hive_table_name = 'default.cr_table_demo_delta'
spark.sql(f"select * from {hive_table_name}").tail(100)

[Row(Part_IsAssembly='0', Part_Weight='4.720362', Part_RealArea='0.5011', Part_ExternalArea='0.5011', Part_Thickness='1.2', Part_Length='769.7837', Part_RectangularArea='0.5831', Part_IsFillerPart='0', Part_MaterialDensity='7.85', Part_RectangularWeight='5.4931', Part_MarkingPerimeter='0.0', Part_ExternalWeight='4.720362', Part_CutPerimeter='3.1167', Part_Width='757.5281', ManufacturingOrder_Quantity='1.0', Part_WorkCenter='Prima Tob Platino CP4000 ()', Part_Material='Stainless Steel', Part_Rotations='1.111111111E49', Part_CreationMethod='DXF', Tenant_ID='3cb9a027-3b3e-4f3d-b9df-d8478cf812c8', Plant_ID='8abe3f95-37f1-4cbc-a152-52e79ed26a65'),
 Row(Part_IsAssembly='0', Part_Weight='2.49787', Part_RealArea='0.1591', Part_ExternalArea='0.2249', Part_Thickness='2.0', Part_Length='379.005', Part_RectangularArea='0.2532', Part_IsFillerPart='0', Part_MaterialDensity='7.85', Part_RectangularWeight='3.9749', Part_MarkingPerimeter='0.0', Part_ExternalWeight='3.5309299999999997', Part_CutPerimete

In [4]:
spark.stop()